In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_csv("C://Users//SNEH//Documents//AMEX//train_data//train_data_with_samplefeatures.csv")
df_batsman = pd.read_csv("C://Users//SNEH//Documents//AMEX//train_data//batsman_level_scorecard.csv")
df_bowler = pd.read_csv("C://Users//SNEH//Documents//AMEX//train_data//bowler_level_scorecard.csv")
df_match = pd.read_csv("C://Users//SNEH//Documents//AMEX//train_data//match_level_scorecard.csv")
df_test = pd.read_csv("C://Users//SNEH//Documents//AMEX//train_data//test_data_with_samplefeatures.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = [99, 1, 11, 23, 55, 66, 77, 88, 100]
y_test = [99, 0, 11, 55, 65, 66, 78]
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
df

In [ ]:
column_required  =[
    "match id",
    "team1_id",
    "team1",
    "team2_id",
    "team2",
    "winner_id",
    "toss winner",
    "toss decision",
    "city",
    "lighting",
    "season",
    "team_count_50runs_last15",
    "team_winp_last5",
    "team1only_avg_runs_last15",
    "team1_winp_team2_last15",
    "ground_avg_runs_last15",
    "team1_roster_ids",
    "team2_roster_ids",
    "match_dt",
    "venue"
]

df_new = df[column_required]
df1 = df_new
#df1 = df_new.dropna()
#df1.reset_index(drop=True, inplace=True)


In [ ]:
df1

In [ ]:

df1["winner"] = None
df1["toss_winner"] = None

for i in range(df1.shape[0]):

    team1_id = df1.iloc[i]["team1_id"]
    team2_id = df1.iloc[i]["team2_id"]
    winner_id = df1.iloc[i]["winner_id"]
    team1_name = df1.iloc[i]["team1"]
    team2_name = df1.iloc[i]["team2"]
    toss_winner = df1.iloc[i]["toss winner"]

    if winner_id == team1_id:
        df1["winner"][i] = 1
    else:
        df1["winner"][i] = 2

    if toss_winner == team1_name:
        df1["toss_winner"][i] = 1
    else:
        df1["toss_winner"][i] = 2
     

In [ ]:
df1['toss decision'] = df1['toss decision'].replace({'field': 0, 'bat': 1})
df1['lighting'] = df1['lighting'].replace({'day/night match': 0, 'day match': 1, 'night match': 2})
df1['season'] = df1['season'].replace({'2021/22': 0, '2022': 1, '2022/23': 2, '2023': 3, '2023/24': 4})

In [ ]:
df1['match_dt'] = pd.to_datetime(df1['match_dt'])
df_batsman['match_dt'] = pd.to_datetime(df_batsman['match_dt'])
df_bowler['match_dt'] = pd.to_datetime(df_bowler['match_dt'])

In [ ]:
df1.isnull().sum()

In [ ]:
df1.dtypes

In [ ]:
df1["team1_bat_win_ratio"] = None

df1["team1_bowl_win_ratio"] = None

df1["team2_bat_win_ratio"] = None
df1["team2_bowl_win_ratio"] = None

df1["team1_win_amount_wickets"] = None
df1["team1_win_amount_runs"] = None

df1["team2_win_amount_wickets"] = None
df1["team2_win_amount_runs"] = None


df1["team1_inning1_avg_runs"] = None
df1["team1_inning1_avg_wickets"] = None
df1["team1_inning2_avg_runs"] = None
df1["team1_inning2_avg_wickets"] = None

df1["team2_inning1_avg_runs"] = None
df1["team2_inning1_avg_wickets"] = None
df1["team2_inning2_avg_runs"] = None
df1["team2_inning2_avg_wickets"] = None


In [ ]:

for i in range(df1.shape[0]):

    
    team1_id = df1.iloc[i]["team1_id"]
    team2_id = df1.iloc[i]["team2_id"]


    filtered_df1 = df_match[df_match['team1_id'] == team1_id]
    filtered_df2 = df_match[df_match['team2_id'] == team1_id]

    filtered_df3 = df_match[df_match['team1_id'] == team2_id]
    filtered_df4 = df_match[df_match['team2_id'] == team2_id]


    matches_bat_1 = [np.nan]
    matches_bat_2 = [np.nan]

    won_bat_1 = [np.nan]
    won_bat_2 = [np.nan]

    matches_bowl_1 = [np.nan]
    matches_bowl_2 = [np.nan]

    won_bowl_1 = [np.nan]
    won_bowl_2 = [np.nan]

    team1_win_amount_wickets_1 = [np.nan]
    team1_win_amount_runs_1 = [np.nan]
    team1_inning1_avg_runs = [np.nan]
    team1_inning1_avg_wickets = [np.nan]
    team1_inning2_avg_runs = [np.nan]
    team1_inning2_avg_wickets = [np.nan]

    team2_win_amount_wickets_1 = [np.nan]
    team2_win_amount_runs_1 = [np.nan]
    team2_inning1_avg_runs = [np.nan]
    team2_inning1_avg_wickets = [np.nan]
    team2_inning2_avg_runs = [np.nan]
    team2_inning2_avg_wickets = [np.nan]



    if filtered_df1.shape[0] > 0:

        team1_name = filtered_df1.iloc[0]['team1']
        #bat win ratio---------------------------------------------------------- 
        filtered_df1_bat1 = filtered_df1[(filtered_df1['toss winner'] == team1_name) & (filtered_df1['toss decision'] == 'bat')]
        filtered_df1_bat2 = filtered_df1[(filtered_df1['toss winner'] != team1_name) & (filtered_df1['toss decision'] == 'field')]

       

        team1_inning1_avg_runs.extend(list(pd.concat([filtered_df1_bat1, filtered_df1_bat2], axis=0, ignore_index=True)['inning1_runs']))
        team1_inning2_avg_wickets.extend(list(pd.concat([filtered_df1_bat1, filtered_df1_bat2], axis=0, ignore_index=True)['inning2_wickets']))

        matches_bat_1.append(filtered_df1_bat1.shape[0] + filtered_df1_bat2.shape[0])
        won_bat_1.append(filtered_df1_bat1[filtered_df1_bat1['winner_id'] == team1_id].shape[0] + filtered_df1_bat2[filtered_df1_bat2['winner_id'] == team1_id].shape[0])

        

        #bowl win ratio--------------------------------------------------------
        filtered_df1_bowl1 = filtered_df1[(filtered_df1['toss winner'] == team1_name) & (filtered_df1['toss decision'] == 'field')]
        filtered_df1_bowl2 = filtered_df1[(filtered_df1['toss winner'] != team1_name) & (filtered_df1['toss decision'] == 'bat')]

      
        team1_inning1_avg_wickets.extend(list(pd.concat([filtered_df1_bowl1, filtered_df1_bowl2], axis=0, ignore_index=True)['inning1_wickets']))
        team1_inning2_avg_runs.extend(list(pd.concat([filtered_df1_bowl1, filtered_df1_bowl2], axis=0, ignore_index=True)['inning2_runs']))

        matches_bowl_1.append(filtered_df1_bowl1.shape[0] + filtered_df1_bowl2.shape[0])
        won_bowl_1.append(filtered_df1_bowl1[filtered_df1_bowl1['winner_id'] == team1_id].shape[0] + filtered_df1_bowl2[filtered_df1_bowl2['winner_id'] == team1_id].shape[0])

        #win amount by wickets and runs--------------------------------------------------

        team1_win_amount_wickets_1.extend(list(filtered_df1[(filtered_df1['winner'] == team1_name) & (filtered_df1['by'] == 'wickets')]['win amount']))
        team1_win_amount_runs_1.extend(list(filtered_df1[(filtered_df1['winner'] == team1_name) & (filtered_df1['by'] == 'runs')]['win amount']))


    if filtered_df2.shape[0] > 0:

        team2_name = filtered_df2.iloc[0]['team2']
        filtered_df2_bat1 = filtered_df2[(filtered_df2['toss winner'] == team2_name) & (filtered_df2['toss decision'] == 'bat')]
        filtered_df2_bat2 = filtered_df2[(filtered_df2['toss winner'] != team2_name) & (filtered_df2['toss decision'] == 'field')]

     
        team1_inning1_avg_runs.extend(list(pd.concat([filtered_df2_bat1, filtered_df2_bat2], axis=0, ignore_index=True)['inning1_runs']))
        team1_inning2_avg_wickets.extend(list(pd.concat([filtered_df2_bat1, filtered_df2_bat2], axis=0, ignore_index=True)['inning2_wickets']))

        
        matches_bat_1.append(filtered_df2_bat1.shape[0] + filtered_df2_bat2.shape[0])
        won_bat_1.append(filtered_df2_bat1[filtered_df2_bat1['winner_id'] == team1_id].shape[0] + filtered_df2_bat2[filtered_df2_bat2['winner_id'] == team1_id].shape[0])
     

        filtered_df2_bowl1 = filtered_df2[(filtered_df2['toss winner'] == team2_name) & (filtered_df2['toss decision'] == 'field')]
        filtered_df2_bowl2 = filtered_df2[(filtered_df2['toss winner'] != team2_name) & (filtered_df2['toss decision'] == 'bat')]

        

        team1_inning1_avg_wickets.extend(list(pd.concat([filtered_df2_bowl1, filtered_df2_bowl2], axis=0, ignore_index=True)['inning1_wickets']))
        team1_inning2_avg_runs.extend(list(pd.concat([filtered_df2_bowl1, filtered_df2_bowl2], axis=0, ignore_index=True)['inning2_runs']))

        matches_bowl_1.append(filtered_df2_bowl1.shape[0] + filtered_df2_bowl2.shape[0])
        won_bowl_1.append(filtered_df2_bowl1[filtered_df2_bowl1['winner_id'] == team1_id].shape[0] + filtered_df2_bowl2[filtered_df2_bowl2['winner_id'] == team1_id].shape[0])

        #win amount by wickets and runs--------------------------------------------------

        team1_win_amount_wickets_1.extend(list(filtered_df2[(filtered_df2['winner'] == team2_name) & (filtered_df2['by'] == 'wickets')]['win amount']))
        team1_win_amount_runs_1.extend(list(filtered_df2[(filtered_df2['winner'] == team2_name) & (filtered_df2['by'] == 'runs')]['win amount']))









    if filtered_df3.shape[0] > 0:

        team3_name = filtered_df3.iloc[0]['team1']
        #bat win ratio---------------------------------------------------------- 
        filtered_df3_bat1 = filtered_df3[(filtered_df3['toss winner'] == team3_name) & (filtered_df3['toss decision'] == 'bat')]
        filtered_df3_bat2 = filtered_df3[(filtered_df3['toss winner'] != team3_name) & (filtered_df3['toss decision'] == 'field')]

        

        team2_inning1_avg_runs.extend(list(pd.concat([filtered_df3_bat1, filtered_df3_bat2], axis=0, ignore_index=True)['inning1_runs']))
        team2_inning2_avg_wickets.extend(list(pd.concat([filtered_df3_bat1, filtered_df3_bat2], axis=0, ignore_index=True)['inning2_wickets']))

        matches_bat_2.append(filtered_df3_bat1.shape[0] + filtered_df3_bat2.shape[0])
        won_bat_2.append(filtered_df3_bat1[filtered_df3_bat1['winner_id'] == team2_id].shape[0] + filtered_df3_bat2[filtered_df3_bat2['winner_id'] == team2_id].shape[0])

        

        #bowl win ratio--------------------------------------------------------
        filtered_df3_bowl1 = filtered_df3[(filtered_df3['toss winner'] == team3_name) & (filtered_df3['toss decision'] == 'field')]
        filtered_df3_bowl2 = filtered_df3[(filtered_df3['toss winner'] != team3_name) & (filtered_df3['toss decision'] == 'bat')]

       

        team2_inning1_avg_wickets.extend(list(pd.concat([filtered_df3_bowl1, filtered_df3_bowl2], axis=0, ignore_index=True)['inning1_wickets']))
        team2_inning2_avg_runs.extend(list(pd.concat([filtered_df3_bowl1, filtered_df3_bowl2], axis=0, ignore_index=True)['inning2_runs']))

        matches_bowl_2.append(filtered_df3_bowl1.shape[0] + filtered_df3_bowl2.shape[0])
        won_bowl_2.append(filtered_df3_bowl1[filtered_df3_bowl1['winner_id'] == team2_id].shape[0] + filtered_df3_bowl2[filtered_df3_bowl2['winner_id'] == team2_id].shape[0])

        #win amount by wickets and runs--------------------------------------------------

        team2_win_amount_wickets_1.extend(list(filtered_df3[(filtered_df3['winner'] == team3_name) & (filtered_df3['by'] == 'wickets')]['win amount']))
        team2_win_amount_runs_1.extend(list(filtered_df3[(filtered_df3['winner'] == team3_name) & (filtered_df3['by'] == 'runs')]['win amount']))


    if filtered_df4.shape[0] > 0:

        team4_name = filtered_df4.iloc[0]['team2']
        filtered_df4_bat1 = filtered_df4[(filtered_df4['toss winner'] == team4_name) & (filtered_df4['toss decision'] == 'bat')]
        filtered_df4_bat2 = filtered_df4[(filtered_df4['toss winner'] != team4_name) & (filtered_df4['toss decision'] == 'field')]

       

        team2_inning1_avg_runs.extend(list(pd.concat([filtered_df4_bat1, filtered_df4_bat2], axis=0, ignore_index=True)['inning1_runs']))
        team2_inning2_avg_wickets.extend(list(pd.concat([filtered_df4_bat1, filtered_df4_bat2], axis=0, ignore_index=True)['inning2_wickets']))

        
        matches_bat_2.append(filtered_df4_bat1.shape[0] + filtered_df4_bat2.shape[0])
        won_bat_2.append(filtered_df4_bat1[filtered_df4_bat1['winner_id'] == team2_id].shape[0] + filtered_df4_bat2[filtered_df4_bat2['winner_id'] == team2_id].shape[0])
     

        filtered_df4_bowl1 = filtered_df4[(filtered_df4['toss winner'] == team4_name) & (filtered_df4['toss decision'] == 'field')]
        filtered_df4_bowl2 = filtered_df4[(filtered_df4['toss winner'] != team4_name) & (filtered_df4['toss decision'] == 'bat')]

       

        team2_inning1_avg_wickets.extend(list(pd.concat([filtered_df4_bowl1, filtered_df4_bowl2], axis=0, ignore_index=True)['inning1_wickets']))
        team2_inning2_avg_runs.extend(list(pd.concat([filtered_df4_bowl1, filtered_df4_bowl2], axis=0, ignore_index=True)['inning2_runs']))

        matches_bowl_2.append(filtered_df4_bowl1.shape[0] + filtered_df4_bowl2.shape[0])
        won_bowl_2.append(filtered_df4_bowl1[filtered_df4_bowl1['winner_id'] == team2_id].shape[0] + filtered_df4_bowl2[filtered_df4_bowl2['winner_id'] == team2_id].shape[0])

        #win amount by wickets and runs--------------------------------------------------

        team2_win_amount_wickets_1.extend(list(filtered_df4[(filtered_df4['winner'] == team4_name) & (filtered_df4['by'] == 'wickets')]['win amount']))
        team2_win_amount_runs_1.extend(list(filtered_df4[(filtered_df4['winner'] == team4_name) & (filtered_df4['by'] == 'runs')]['win amount']))
    




    df1["team1_bat_win_ratio"][i] = np.nansum(won_bat_1)/np.nansum(matches_bat_1)
    df1["team1_bowl_win_ratio"][i] = np.nansum(won_bowl_1)/np.nansum(matches_bowl_1)

    df1["team1_win_amount_wickets"][i] = np.nanmean(team1_win_amount_wickets_1)
    df1["team1_win_amount_runs"][i] = np.nanmean(team1_win_amount_runs_1)

    df1["team1_inning1_avg_runs"][i] = np.nanmean(team1_inning1_avg_runs)
    df1["team1_inning1_avg_wickets"][i] = np.nanmean(team1_inning1_avg_wickets)
    df1["team1_inning2_avg_runs"][i] = np.nanmean(team1_inning2_avg_runs)
    df1["team1_inning2_avg_wickets"][i] = np.nanmean(team1_inning2_avg_wickets)

    df1["team2_bat_win_ratio"][i] = np.nansum(won_bat_2)/np.nansum(matches_bat_2)
    df1["team2_bowl_win_ratio"][i] = np.nansum(won_bowl_2)/np.nansum(matches_bowl_2)

    df1["team2_win_amount_wickets"][i] = np.nanmean(team2_win_amount_wickets_1)
    df1["team2_win_amount_runs"][i] = np.nanmean(team2_win_amount_runs_1)

    df1["team2_inning1_avg_runs"][i] = np.nanmean(team2_inning1_avg_runs)
    df1["team2_inning1_avg_wickets"][i] = np.nanmean(team2_inning1_avg_wickets)
    df1["team2_inning2_avg_runs"][i] = np.nanmean(team2_inning2_avg_runs)
    df1["team2_inning2_avg_wickets"][i] = np.nanmean(team2_inning2_avg_wickets)


In [ ]:
df1

In [ ]:
df1["team1_top3_bat_strike_rate"] = None
df1["team2_top3_bat_strike_rate"] = None

df1["team1_top3_bat_run"] = None
df1["team2_top3_bat_run"] = None

df1["team1_top3_bat_6s+4s"] = None
df1["team2_top3_bat_6s+4s"] = None

df1["team2_above30_consistancy"] = None
df1["team1_above30_consistancy"] = None

df1["team1_top3_bowl_dots"] = None
df1["team2_top3_bowl_dots"] = None

df1["team1_top3_wickets/ball"] = None
df1["team2_top3_wickets/ball"] = None

df1["team1_top3_avg_economy"] = None
df1["team2_top3_avg_economy"] = None

df1["team1_top3_bowl_6s+4s"] = None
df1["team2_top3_bowl_6s+4s"] = None

df1["team1_top3_wide_noball"] = None
df1["team2_top3_wide_noball"] = None

df1["player_of_the_match_team1"] = None
df1["player_of_the_match_team2"] = None

df1["team1_player_wickets"] = None
df1["team2_player_wickets"] = None

for i in range(df1.shape[0]):

    match_id = df1.iloc[i]["match id"]
    match_date = df1.iloc[i]["match_dt"]
    team1_players = list(map(lambda x: float(x), list(df1["team1_roster_ids"][i].split(":"))))
    team2_players = list(map(lambda x: float(x), list(df1["team2_roster_ids"][i].split(":"))))

    team1_avg_runs = []
    team2_avg_runs = []
    team1_avg_strike_rate = []
    team2_avg_strike_rate = []
    team1_total_6s_4s = []
    team2_total_6s_4s = []
    team1_above30_consistancy = []
    team2_above30_consistancy = []
    team1_dots = []
    team2_dots = []
    team1_wicketsbyballs = []
    team2_wicketsbyballs = []
    team1_avg_economy = []
    team2_avg_economy = []
    team1_total_6s_4s_bowl = []
    team1_total_wide_noball = []
    team2_total_6s_4s_bowl = []
    team2_total_wide_noball = []
    team1_total_player_of_match = []
    team2_total_player_of_match = []
    team2_player_wickets = []
    team1_player_wickets = []




    for player in team1_players:

        player_strike_rate = []
        player_runs = []
        player_6s_4s = []
        player_economy = []
        player_wicketsbyball = []
        player_wickets = []
        player_dots = []
        player_6s_4s_bowl = []
        player_wides_noballs = []


        team1_total_player_of_match.append(df_match[df_match['player_of_the_match_id'] == player].shape[0])



        filtered_df = df_batsman[df_batsman['batsman_id'] == player]

        if filtered_df.shape[0] > 0:
 #           filtered_df = filtered_df[filtered_df['match_dt'] < match_date]

 #           if filtered_df.shape[0] > 0:

            for j in range(filtered_df.shape[0]):
                player_6s_4s.append(list(filtered_df["Sixes"])[j])
                player_6s_4s.append(list(filtered_df["Fours"])[j])
                player_strike_rate.append(list(filtered_df["strike_rate"])[j])
                player_runs.append(list(filtered_df["runs"])[j])
                

        filtered_df1 = df_bowler[df_bowler['bowler_id'] == player]

        if filtered_df1.shape[0] > 0:
#            filtered_df1 = filtered_df1[filtered_df1['match_dt'] < match_date]

#            if filtered_df1.shape[0] > 0:

            for k in range(filtered_df1.shape[0]):
                player_economy.append(list(filtered_df1["economy"])[k])
                player_dots.append((list(filtered_df1["dots"])[k]*10)/list(filtered_df1["balls_bowled"])[k])
                player_wicketsbyball.append((list(filtered_df1["wicket_count"])[k]*10)/list(filtered_df1["balls_bowled"])[k])
                player_wickets.append(list(filtered_df1["wicket_count"])[k])
                player_6s_4s_bowl.append(list(filtered_df1["Fours"])[k])
                player_6s_4s_bowl.append(list(filtered_df1["Sixes"])[k])
                player_wides_noballs.append(list(filtered_df1["wides"])[k])
                player_wides_noballs.append(list(filtered_df1["noballs"])[k])

        if len(player_strike_rate) != 0:
            player_avg_strike_rate = np.nanmean(np.array(player_strike_rate))
            team1_avg_strike_rate.append(player_avg_strike_rate)
        
        if len(player_runs) != 0:
            player_avg_runs = np.nanmean(np.array(player_runs))
            team1_avg_runs.append(player_avg_runs)

            new_runs = [30 if x>29 else x for x in player_runs]
            cv1 =[x - 30 for x in new_runs]
            cv1 = [x*x for x in cv1]
            cv1 = np.sqrt(np.nanmean(np.array(cv1)))
            team1_above30_consistancy.append(cv1)


        player_total_6s4s = np.nansum(np.array(player_6s_4s))
        team1_total_6s_4s.append(player_total_6s4s)

        player_total_6s4s_bowl = np.nansum(np.array(player_6s_4s_bowl))
        team1_total_6s_4s_bowl.append(player_total_6s4s_bowl)

        player_total_wide_noball = np.nansum(np.array(player_wides_noballs))
        team1_total_wide_noball.append(player_total_wide_noball)

        player_total_wickets = np.nansum(np.array(player_wickets))
        team1_player_wickets.append(player_total_wickets)

        if len(player_economy) != 0:
            player_avg_economy = np.nanmean(np.array(player_economy))
            team1_avg_economy.append(player_avg_economy)

        if len(player_wicketsbyball) != 0:
            player_avg_wicketsbyball = np.nanmean(np.array(player_wicketsbyball))
            team1_wicketsbyballs.append(player_avg_wicketsbyball)

        if len(player_dots) != 0:
            player_avg_dotsbyball = np.nanmean(np.array(player_dots))
            team1_dots.append(player_avg_dotsbyball)



    for player in team2_players:

        player_strike_rate = []
        player_runs = []
        player_6s_4s = []
        player_economy = []
        player_wicketsbyball = []
        player_wickets = []
        player_dots = []
        player_6s_4s_bowl = []
        player_wides_noballs = []

        team2_total_player_of_match.append(df_match[df_match['player_of_the_match_id'] == player].shape[0])

        filtered_df = df_batsman[df_batsman['batsman_id'] == player]

        if filtered_df.shape[0] > 0:
    #        filtered_df = filtered_df[filtered_df['match_dt'] < match_date]

    #        if filtered_df.shape[0] > 0:

            for j in range(filtered_df.shape[0]):
                player_6s_4s.append(list(filtered_df["Sixes"])[j])
                player_6s_4s.append(list(filtered_df["Fours"])[j])
                player_strike_rate.append(list(filtered_df["strike_rate"])[j])
                player_runs.append(list(filtered_df["runs"])[j])

            

        filtered_df1 = df_bowler[df_bowler['bowler_id'] == player]

        if filtered_df1.shape[0] > 0:
        #    filtered_df1 = filtered_df1[filtered_df1['match_dt'] < match_date]

        #    if filtered_df1.shape[0] > 0:

            for k in range(filtered_df1.shape[0]):
                player_economy.append(list(filtered_df1["economy"])[k])
                player_dots.append((list(filtered_df1["dots"])[k]*10)/list(filtered_df1["balls_bowled"])[k])
                player_wicketsbyball.append((list(filtered_df1["wicket_count"])[k]*10)/list(filtered_df1["balls_bowled"])[k])
                player_wickets.append(list(filtered_df1["wicket_count"])[k])
                player_6s_4s_bowl.append(list(filtered_df1["Fours"])[k])
                player_6s_4s_bowl.append(list(filtered_df1["Sixes"])[k])
                player_wides_noballs.append(list(filtered_df1["wides"])[k])
                player_wides_noballs.append(list(filtered_df1["noballs"])[k])

        if len(player_strike_rate) != 0:
            player_avg_strike_rate = np.nanmean(np.array(player_strike_rate))
            team2_avg_strike_rate.append(player_avg_strike_rate)
        
        if len(player_runs) != 0:
            player_avg_runs = np.nanmean(np.array(player_runs))
            team2_avg_runs.append(player_avg_runs)

            new_runs = [30 if x>29 else x for x in player_runs]
            cv1 =[x - 30 for x in new_runs]
            cv1 = [x*x for x in cv1]
            cv1 = np.sqrt(np.nanmean(np.array(cv1)))
            team2_above30_consistancy.append(cv1)


        player_total_6s4s = np.nansum(np.array(player_6s_4s))
        team2_total_6s_4s.append(player_total_6s4s)

        player_total_6s4s_bowl = np.nansum(np.array(player_6s_4s_bowl))
        team2_total_6s_4s_bowl.append(player_total_6s4s_bowl)

        player_total_wide_noball = np.nansum(np.array(player_wides_noballs))
        team2_total_wide_noball.append(player_total_wide_noball)

        player_total_wickets = np.nansum(np.array(player_wickets))
        team2_player_wickets.append(player_total_wickets)

        if len(player_economy) != 0:
            player_avg_economy = np.nanmean(np.array(player_economy))
            team2_avg_economy.append(player_avg_economy)

        if len(player_wicketsbyball) != 0:
            player_avg_wicketsbyball = np.nanmean(np.array(player_wicketsbyball))
            team2_wicketsbyballs.append(player_avg_wicketsbyball)

        if len(player_dots) != 0:
            player_avg_dotsbyball = np.nanmean(np.array(player_dots))
            team2_dots.append(player_avg_dotsbyball)

    team1_avg_strike_rate = [x-125 if x>125 else 0 for x in team1_avg_strike_rate]
    team1_avg_strike_rate = [x*x for x in team1_avg_strike_rate]

    df1["team1_top3_bat_strike_rate"][i] = np.sqrt(np.nanmean(np.array(team1_avg_strike_rate)))
#--------------------------------------------------------------------------------------------
    team2_avg_strike_rate = [x-125 if x>125 else 0 for x in team2_avg_strike_rate]
    team2_avg_strike_rate = [x*x for x in team2_avg_strike_rate]

    df1["team2_top3_bat_strike_rate"][i] = np.sqrt(np.nanmean(np.array(team2_avg_strike_rate)))
#------------------------------------------------------------------------------------

    team1_avg_runs = [x-25 if x>25 else 0 for x in team1_avg_runs]
    team1_avg_runs = [x*x for x in team1_avg_runs]

    df1["team1_top3_bat_run"][i] = np.sqrt(np.nanmean(np.array(team1_avg_runs)))

#------------------------------------------------------------------------------------------

    team2_avg_runs = [x-25 if x>25 else 0 for x in team2_avg_runs]
    team2_avg_runs = [x*x for x in team2_avg_runs]

    df1["team2_top3_bat_run"][i] = np.sqrt(np.nanmean(np.array(team2_avg_runs)))

#-----------------------------------------------------------------------------------------
    df1["team1_top3_bat_6s+4s"][i] = np.nanmean(np.array(sorted(team1_total_6s_4s, reverse=True)))
    df1["team2_top3_bat_6s+4s"][i] = np.nanmean(np.array(sorted(team2_total_6s_4s, reverse=True)))

    df1["team1_top3_bowl_6s+4s"][i] = np.nanmean(np.array(sorted(team1_total_6s_4s_bowl, reverse=False)))
    df1["team2_top3_bowl_6s+4s"][i] = np.nanmean(np.array(sorted(team2_total_6s_4s_bowl, reverse=False)))

    df1["team1_top3_wide_noball"][i] = np.nanmean(np.array(sorted(team1_total_wide_noball, reverse=False)))
    df1["team2_top3_wide_noball"][i] = np.nanmean(np.array(sorted(team2_total_wide_noball, reverse=False)))


#------------------------------------------------------------------------------------
    if len(team1_above30_consistancy) > 2:
        df1["team1_above30_consistancy"][i] = np.nanmean(np.array(sorted(team1_above30_consistancy, reverse=False)))
    else:
        df1["team1_above30_consistancy"][i] = np.nanmean(np.array(team1_above30_consistancy))
#------------------------------------------------------------------------------------------
    if len(team2_above30_consistancy) > 2:
        df1["team2_above30_consistancy"][i] = np.nanmean(np.array(sorted(team2_above30_consistancy, reverse=False)))
    else:
        df1["team2_above30_consistancy"][i] = np.nanmean(np.array(team2_above30_consistancy))

#-------------------------------------------------------------------------------------------------------------------------
    team1_avg_economy = [0 if x>7.5 else x for x in team1_avg_economy]
    team1_avg_economy = [x*x for x in team1_avg_economy]

    df1["team1_top3_avg_economy"][i] = np.sqrt(np.nanmean(np.array(team1_avg_economy)))

#----------------------------------------------------------------------------------------------------------
    team2_avg_economy = [0 if x>7.5 else x for x in team2_avg_economy]
    team2_avg_economy = [x*x for x in team2_avg_economy]

    df1["team2_top3_avg_economy"][i] = np.sqrt(np.nanmean(np.array(team2_avg_economy)))


#-------------------------------------------------------------------------------------------
    if len(team2_wicketsbyballs) > 2:
        df1["team2_top3_wickets/ball"][i] = np.nanmean(np.array(sorted(team2_wicketsbyballs, reverse=True)))
    else:
        df1["team2_top3_wickets/ball"][i] = np.nanmean(np.array(team2_wicketsbyballs))

#-----------------------------------------------------------------------------------------------------------------
    if len(team1_wicketsbyballs) > 2:
        df1["team1_top3_wickets/ball"][i] = np.nanmean(np.array(sorted(team1_wicketsbyballs, reverse=True)))
    else:
        df1["team1_top3_wickets/ball"][i] = np.nanmean(np.array(team1_wicketsbyballs))

#---------------------------------------------------------------------------------------------------------
    if len(team1_dots) > 2:
        df1["team1_top3_bowl_dots"][i] = np.nanmean(np.array(sorted(team1_dots, reverse=True)))
    else:
        df1["team1_top3_bowl_dots"][i] = np.nanmean(np.array(team1_dots))

#-------------------------------------------------------------------------------------------------------
    if len(team2_dots) > 2:
        df1["team2_top3_bowl_dots"][i] = np.nanmean(np.array(sorted(team2_dots, reverse=True)))
    else:
        df1["team2_top3_bowl_dots"][i] = np.nanmean(np.array(team2_dots))

#--------------------------------------------------------------------------------------------------------

    df1["player_of_the_match_team1"][i] = np.nansum(np.array(team1_total_player_of_match))
    df1["player_of_the_match_team2"][i] = np.nansum(np.array(team2_total_player_of_match))

    df1["team1_player_wickets"][i] = np.nansum(np.array(team1_player_wickets))
    df1["team2_player_wickets"][i] = np.nansum(np.array(team2_player_wickets))



In [ ]:
df1

In [ ]:
df_train = df1.drop(["team1", "team2", "toss winner", "team1_roster_ids", "team2_roster_ids"], axis=1)

In [ ]:
df_train

In [ ]:
df_train = df_train.drop('match id', axis= 1)

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.to_csv("train_all_feat.csv", index= False)

In [ ]:
df_train = df_train.drop(["team1_id", "team2_id", "winner_id"], axis=1)

In [ ]:
df_train["city"]

In [ ]:
labels = list(df_train['winner'])
venue = list(df_train['venue'])
city = list(df_train['city'])
match_date = list(df_train['match_dt'])
features = df_train
features = features.drop(["winner", "venue", "city", "match_dt"], axis=1)

In [ ]:
# Initialize the IterativeImputer with RandomForestRegressor
imputer = IterativeImputer(estimator=RandomForestRegressor(), random_state=0)

# Fit the imputer and transform the features
imputed_features = imputer.fit_transform(features)

# Convert the result back to a DataFrame
features_imputed = pd.DataFrame(imputed_features, columns=features.columns)

In [ ]:
features_imputed

In [ ]:
features_imputed["winner"] = labels
features_imputed["venue"] = venue
features_imputed["city"] = city
features_imputed["match_dt"] = match_date

In [ ]:
features_imputed

In [ ]:
for i in range(features_imputed.shape[0]):

    features_imputed["match_dt"][i] = features_imputed["match_dt"][i].month

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder


label_encoder = LabelEncoder()
features_imputed["venue"] = label_encoder.fit_transform(features_imputed["venue"])
features_imputed["city"] = label_encoder.fit_transform(features_imputed["city"])

In [ ]:
features_imputed

In [ ]:
features_imputed.to_csv("Train_imputed_all_feat_labeled222.csv", index= False)

In [ ]:
df_train

In [ ]:
df = pd.read_csv("")

In [ ]:
df_sample = pd.read_csv("submission_template_file1.csv")
df_test = pd.read_csv("C://Users//SNEH//Documents//AMEX//train_data//test_data_with_samplefeatures.csv")
df_tra = pd.read_csv("C://Users//SNEH//Documents//AMEX//train_data//train_data_with_samplefeatures.csv")
df_match = pd.read_csv("C://Users//SNEH//Documents//AMEX//train_data//match_level_scorecard.csv")

In [ ]:
df_test

In [ ]:
match_id = df_sample['match id']


In [ ]:
#df_test["winner"] = None
df_test["toss_winner"] = None

for i in range(df_test.shape[0]):

    team1_id = df_test.iloc[i]["team1_id"]
    team2_id = df_test.iloc[i]["team2_id"]
    #winner_id = df_test.iloc[i]["winner_id"]
    team1_name = df_test.iloc[i]["team1"]
    team2_name = df_test.iloc[i]["team2"]
    toss_winner = df_test.iloc[i]["toss winner"]

    if toss_winner == team1_name:
        df_test["toss_winner"][i] = 1
    else:
        df_test["toss_winner"][i] = 2

In [ ]:
df_test['toss decision'] = df_test['toss decision'].replace({'field': 0, 'bat': 1})
df_test['lighting'] = df_test['lighting'].replace({'day/night match': 0, 'day match': 1, 'night match': 2})
df_test['season'] = df_test['season'].replace({'2021/22': 0, '2022': 1, '2022/23': 2, '2023': 3, '2023/24': 4})

In [ ]:
df_test = df_test.drop(["team1", "team2", "toss winner", "series_name", "ground_id"], axis=1)

In [ ]:
df_test.head()

In [ ]:
df_tra["toss_winner"] = None

for i in range(df_tra.shape[0]):

    team1_id = df_tra.iloc[i]["team1_id"]
    team2_id = df_tra.iloc[i]["team2_id"]
    #winner_id = df_test.iloc[i]["winner_id"]
    team1_name = df_tra.iloc[i]["team1"]
    team2_name = df_tra.iloc[i]["team2"]
    toss_winner = df_tra.iloc[i]["toss winner"]

    if toss_winner == team1_name:
        df_tra["toss_winner"][i] = 1
    else:
        df_tra["toss_winner"][i] = 2

In [ ]:
df_tra['toss decision'] = df_tra['toss decision'].replace({'field': 0, 'bat': 1})
df_tra['lighting'] = df_tra['lighting'].replace({'day/night match': 0, 'day match': 1, 'night match': 2})
df_tra['season'] = df_tra['season'].replace({'2021/22': 0, '2022': 1, '2022/23': 2, '2023': 3, '2023/24': 4})

In [ ]:
df_tra = df_tra.drop(["team1", "team2", "toss winner", "series_name", "winner", "winner_id", "ground_id"], axis=1)

In [ ]:
result = pd.concat([df_test, df_tra], axis=0)

In [ ]:
float(list(result["team1_roster_ids"])[0].split(":")[0])

In [ ]:
df_batsman = pd.read_csv("C://Users//SNEH//Documents//AMEX//train_data//batsman_level_scorecard.csv")

In [ ]:
result

In [ ]:
result["team1_bat_win_ratio"] = None

result["team1_bowl_win_ratio"] = None

result["team2_bat_win_ratio"] = None
result["team2_bowl_win_ratio"] = None

result["team1_win_amount_wickets"] = None
result["team1_win_amount_runs"] = None

result["team2_win_amount_wickets"] = None
result["team2_win_amount_runs"] = None


result["team1_inning1_avg_runs"] = None
result["team1_inning1_avg_wickets"] = None
result["team1_inning2_avg_runs"] = None
result["team1_inning2_avg_wickets"] = None

result["team2_inning1_avg_runs"] = None
result["team2_inning1_avg_wickets"] = None
result["team2_inning2_avg_runs"] = None
result["team2_inning2_avg_wickets"] = None

In [ ]:
for i in range(result.shape[0]):

    
    team1_id = result.iloc[i]["team1_id"]
    team2_id = result.iloc[i]["team2_id"]


    filtered_df1 = df_match[df_match['team1_id'] == team1_id]
    filtered_df2 = df_match[df_match['team2_id'] == team1_id]

    filtered_df3 = df_match[df_match['team1_id'] == team2_id]
    filtered_df4 = df_match[df_match['team2_id'] == team2_id]


    matches_bat_1 = [np.nan]
    matches_bat_2 = [np.nan]

    won_bat_1 = [np.nan]
    won_bat_2 = [np.nan]

    matches_bowl_1 = [np.nan]
    matches_bowl_2 = [np.nan]

    won_bowl_1 = [np.nan]
    won_bowl_2 = [np.nan]

    team1_win_amount_wickets_1 = [np.nan]
    team1_win_amount_runs_1 = [np.nan]
    team1_inning1_avg_runs = [np.nan]
    team1_inning1_avg_wickets = [np.nan]
    team1_inning2_avg_runs = [np.nan]
    team1_inning2_avg_wickets = [np.nan]

    team2_win_amount_wickets_1 = [np.nan]
    team2_win_amount_runs_1 = [np.nan]
    team2_inning1_avg_runs = [np.nan]
    team2_inning1_avg_wickets = [np.nan]
    team2_inning2_avg_runs = [np.nan]
    team2_inning2_avg_wickets = [np.nan]



    if filtered_df1.shape[0] > 0:

        team1_name = filtered_df1.iloc[0]['team1']
        #bat win ratio---------------------------------------------------------- 
        filtered_df1_bat1 = filtered_df1[(filtered_df1['toss winner'] == team1_name) & (filtered_df1['toss decision'] == 'bat')]
        filtered_df1_bat2 = filtered_df1[(filtered_df1['toss winner'] != team1_name) & (filtered_df1['toss decision'] == 'field')]

       

        team1_inning1_avg_runs.extend(list(pd.concat([filtered_df1_bat1, filtered_df1_bat2], axis=0, ignore_index=True)['inning1_runs']))
        team1_inning2_avg_wickets.extend(list(pd.concat([filtered_df1_bat1, filtered_df1_bat2], axis=0, ignore_index=True)['inning2_wickets']))

        matches_bat_1.append(filtered_df1_bat1.shape[0] + filtered_df1_bat2.shape[0])
        won_bat_1.append(filtered_df1_bat1[filtered_df1_bat1['winner_id'] == team1_id].shape[0] + filtered_df1_bat2[filtered_df1_bat2['winner_id'] == team1_id].shape[0])

        

        #bowl win ratio--------------------------------------------------------
        filtered_df1_bowl1 = filtered_df1[(filtered_df1['toss winner'] == team1_name) & (filtered_df1['toss decision'] == 'field')]
        filtered_df1_bowl2 = filtered_df1[(filtered_df1['toss winner'] != team1_name) & (filtered_df1['toss decision'] == 'bat')]

      
        team1_inning1_avg_wickets.extend(list(pd.concat([filtered_df1_bowl1, filtered_df1_bowl2], axis=0, ignore_index=True)['inning1_wickets']))
        team1_inning2_avg_runs.extend(list(pd.concat([filtered_df1_bowl1, filtered_df1_bowl2], axis=0, ignore_index=True)['inning2_runs']))

        matches_bowl_1.append(filtered_df1_bowl1.shape[0] + filtered_df1_bowl2.shape[0])
        won_bowl_1.append(filtered_df1_bowl1[filtered_df1_bowl1['winner_id'] == team1_id].shape[0] + filtered_df1_bowl2[filtered_df1_bowl2['winner_id'] == team1_id].shape[0])

        #win amount by wickets and runs--------------------------------------------------

        team1_win_amount_wickets_1.extend(list(filtered_df1[(filtered_df1['winner'] == team1_name) & (filtered_df1['by'] == 'wickets')]['win amount']))
        team1_win_amount_runs_1.extend(list(filtered_df1[(filtered_df1['winner'] == team1_name) & (filtered_df1['by'] == 'runs')]['win amount']))


    if filtered_df2.shape[0] > 0:

        team2_name = filtered_df2.iloc[0]['team2']
        filtered_df2_bat1 = filtered_df2[(filtered_df2['toss winner'] == team2_name) & (filtered_df2['toss decision'] == 'bat')]
        filtered_df2_bat2 = filtered_df2[(filtered_df2['toss winner'] != team2_name) & (filtered_df2['toss decision'] == 'field')]

     
        team1_inning1_avg_runs.extend(list(pd.concat([filtered_df2_bat1, filtered_df2_bat2], axis=0, ignore_index=True)['inning1_runs']))
        team1_inning2_avg_wickets.extend(list(pd.concat([filtered_df2_bat1, filtered_df2_bat2], axis=0, ignore_index=True)['inning2_wickets']))

        
        matches_bat_1.append(filtered_df2_bat1.shape[0] + filtered_df2_bat2.shape[0])
        won_bat_1.append(filtered_df2_bat1[filtered_df2_bat1['winner_id'] == team1_id].shape[0] + filtered_df2_bat2[filtered_df2_bat2['winner_id'] == team1_id].shape[0])
     

        filtered_df2_bowl1 = filtered_df2[(filtered_df2['toss winner'] == team2_name) & (filtered_df2['toss decision'] == 'field')]
        filtered_df2_bowl2 = filtered_df2[(filtered_df2['toss winner'] != team2_name) & (filtered_df2['toss decision'] == 'bat')]

        

        team1_inning1_avg_wickets.extend(list(pd.concat([filtered_df2_bowl1, filtered_df2_bowl2], axis=0, ignore_index=True)['inning1_wickets']))
        team1_inning2_avg_runs.extend(list(pd.concat([filtered_df2_bowl1, filtered_df2_bowl2], axis=0, ignore_index=True)['inning2_runs']))

        matches_bowl_1.append(filtered_df2_bowl1.shape[0] + filtered_df2_bowl2.shape[0])
        won_bowl_1.append(filtered_df2_bowl1[filtered_df2_bowl1['winner_id'] == team1_id].shape[0] + filtered_df2_bowl2[filtered_df2_bowl2['winner_id'] == team1_id].shape[0])

        #win amount by wickets and runs--------------------------------------------------

        team1_win_amount_wickets_1.extend(list(filtered_df2[(filtered_df2['winner'] == team2_name) & (filtered_df2['by'] == 'wickets')]['win amount']))
        team1_win_amount_runs_1.extend(list(filtered_df2[(filtered_df2['winner'] == team2_name) & (filtered_df2['by'] == 'runs')]['win amount']))









    if filtered_df3.shape[0] > 0:

        team3_name = filtered_df3.iloc[0]['team1']
        #bat win ratio---------------------------------------------------------- 
        filtered_df3_bat1 = filtered_df3[(filtered_df3['toss winner'] == team3_name) & (filtered_df3['toss decision'] == 'bat')]
        filtered_df3_bat2 = filtered_df3[(filtered_df3['toss winner'] != team3_name) & (filtered_df3['toss decision'] == 'field')]

        

        team2_inning1_avg_runs.extend(list(pd.concat([filtered_df3_bat1, filtered_df3_bat2], axis=0, ignore_index=True)['inning1_runs']))
        team2_inning2_avg_wickets.extend(list(pd.concat([filtered_df3_bat1, filtered_df3_bat2], axis=0, ignore_index=True)['inning2_wickets']))

        matches_bat_2.append(filtered_df3_bat1.shape[0] + filtered_df3_bat2.shape[0])
        won_bat_2.append(filtered_df3_bat1[filtered_df3_bat1['winner_id'] == team2_id].shape[0] + filtered_df3_bat2[filtered_df3_bat2['winner_id'] == team2_id].shape[0])

        

        #bowl win ratio--------------------------------------------------------
        filtered_df3_bowl1 = filtered_df3[(filtered_df3['toss winner'] == team3_name) & (filtered_df3['toss decision'] == 'field')]
        filtered_df3_bowl2 = filtered_df3[(filtered_df3['toss winner'] != team3_name) & (filtered_df3['toss decision'] == 'bat')]

       

        team2_inning1_avg_wickets.extend(list(pd.concat([filtered_df3_bowl1, filtered_df3_bowl2], axis=0, ignore_index=True)['inning1_wickets']))
        team2_inning2_avg_runs.extend(list(pd.concat([filtered_df3_bowl1, filtered_df3_bowl2], axis=0, ignore_index=True)['inning2_runs']))

        matches_bowl_2.append(filtered_df3_bowl1.shape[0] + filtered_df3_bowl2.shape[0])
        won_bowl_2.append(filtered_df3_bowl1[filtered_df3_bowl1['winner_id'] == team2_id].shape[0] + filtered_df3_bowl2[filtered_df3_bowl2['winner_id'] == team2_id].shape[0])

        #win amount by wickets and runs--------------------------------------------------

        team2_win_amount_wickets_1.extend(list(filtered_df3[(filtered_df3['winner'] == team3_name) & (filtered_df3['by'] == 'wickets')]['win amount']))
        team2_win_amount_runs_1.extend(list(filtered_df3[(filtered_df3['winner'] == team3_name) & (filtered_df3['by'] == 'runs')]['win amount']))


    if filtered_df4.shape[0] > 0:

        team4_name = filtered_df4.iloc[0]['team2']
        filtered_df4_bat1 = filtered_df4[(filtered_df4['toss winner'] == team4_name) & (filtered_df4['toss decision'] == 'bat')]
        filtered_df4_bat2 = filtered_df4[(filtered_df4['toss winner'] != team4_name) & (filtered_df4['toss decision'] == 'field')]

       

        team2_inning1_avg_runs.extend(list(pd.concat([filtered_df4_bat1, filtered_df4_bat2], axis=0, ignore_index=True)['inning1_runs']))
        team2_inning2_avg_wickets.extend(list(pd.concat([filtered_df4_bat1, filtered_df4_bat2], axis=0, ignore_index=True)['inning2_wickets']))

        
        matches_bat_2.append(filtered_df4_bat1.shape[0] + filtered_df4_bat2.shape[0])
        won_bat_2.append(filtered_df4_bat1[filtered_df4_bat1['winner_id'] == team2_id].shape[0] + filtered_df4_bat2[filtered_df4_bat2['winner_id'] == team2_id].shape[0])
     

        filtered_df4_bowl1 = filtered_df4[(filtered_df4['toss winner'] == team4_name) & (filtered_df4['toss decision'] == 'field')]
        filtered_df4_bowl2 = filtered_df4[(filtered_df4['toss winner'] != team4_name) & (filtered_df4['toss decision'] == 'bat')]

       

        team2_inning1_avg_wickets.extend(list(pd.concat([filtered_df4_bowl1, filtered_df4_bowl2], axis=0, ignore_index=True)['inning1_wickets']))
        team2_inning2_avg_runs.extend(list(pd.concat([filtered_df4_bowl1, filtered_df4_bowl2], axis=0, ignore_index=True)['inning2_runs']))

        matches_bowl_2.append(filtered_df4_bowl1.shape[0] + filtered_df4_bowl2.shape[0])
        won_bowl_2.append(filtered_df4_bowl1[filtered_df4_bowl1['winner_id'] == team2_id].shape[0] + filtered_df4_bowl2[filtered_df4_bowl2['winner_id'] == team2_id].shape[0])

        #win amount by wickets and runs--------------------------------------------------

        team2_win_amount_wickets_1.extend(list(filtered_df4[(filtered_df4['winner'] == team4_name) & (filtered_df4['by'] == 'wickets')]['win amount']))
        team2_win_amount_runs_1.extend(list(filtered_df4[(filtered_df4['winner'] == team4_name) & (filtered_df4['by'] == 'runs')]['win amount']))
    




    result["team1_bat_win_ratio"][i] = np.nansum(won_bat_1)/np.nansum(matches_bat_1)
    result["team1_bowl_win_ratio"][i] = np.nansum(won_bowl_1)/np.nansum(matches_bowl_1)

    result["team1_win_amount_wickets"][i] = np.nanmean(team1_win_amount_wickets_1)
    result["team1_win_amount_runs"][i] = np.nanmean(team1_win_amount_runs_1)

    result["team1_inning1_avg_runs"][i] = np.nanmean(team1_inning1_avg_runs)
    result["team1_inning1_avg_wickets"][i] = np.nanmean(team1_inning1_avg_wickets)
    result["team1_inning2_avg_runs"][i] = np.nanmean(team1_inning2_avg_runs)
    result["team1_inning2_avg_wickets"][i] = np.nanmean(team1_inning2_avg_wickets)

    result["team2_bat_win_ratio"][i] = np.nansum(won_bat_2)/np.nansum(matches_bat_2)
    result["team2_bowl_win_ratio"][i] = np.nansum(won_bowl_2)/np.nansum(matches_bowl_2)

    result["team2_win_amount_wickets"][i] = np.nanmean(team2_win_amount_wickets_1)
    result["team2_win_amount_runs"][i] = np.nanmean(team2_win_amount_runs_1)

    result["team2_inning1_avg_runs"][i] = np.nanmean(team2_inning1_avg_runs)
    result["team2_inning1_avg_wickets"][i] = np.nanmean(team2_inning1_avg_wickets)
    result["team2_inning2_avg_runs"][i] = np.nanmean(team2_inning2_avg_runs)
    result["team2_inning2_avg_wickets"][i] = np.nanmean(team2_inning2_avg_wickets)


In [ ]:
result['match_dt'] = pd.to_datetime(result['match_dt'])
df_batsman['match_dt'] = pd.to_datetime(df_batsman['match_dt'])

In [ ]:
list(result["team1_roster_ids"].iloc[0].split(":"))


In [ ]:
result["team1_top3_bat_strike_rate"] = None
result["team2_top3_bat_strike_rate"] = None

result["team1_top3_bat_run"] = None
result["team2_top3_bat_run"] = None

result["team1_top3_bat_6s+4s"] = None
result["team2_top3_bat_6s+4s"] = None

result["team2_above30_consistancy"] = None
result["team1_above30_consistancy"] = None

result["team1_top3_bowl_dots"] = None
result["team2_top3_bowl_dots"] = None

result["team1_top3_wickets/ball"] = None
result["team2_top3_wickets/ball"] = None

result["team1_top3_avg_economy"] = None
result["team2_top3_avg_economy"] = None

result["team1_top3_bowl_6s+4s"] = None
result["team2_top3_bowl_6s+4s"] = None

result["team1_top3_wide_noball"] = None
result["team2_top3_wide_noball"] = None

result["player_of_the_match_team1"] = None
result["player_of_the_match_team2"] = None

result["team1_player_wickets"] = None
result["team2_player_wickets"] = None

for i in range(result.shape[0]):

    match_id = result.iloc[i]["match id"]
    match_date = result.iloc[i]["match_dt"]
    team1_players = list(map(lambda x: float(x), list(result["team1_roster_ids"].iloc[i].split(":"))))
    team2_players = list(map(lambda x: float(x), list(result["team2_roster_ids"].iloc[i].split(":"))))

    team1_avg_runs = []
    team2_avg_runs = []
    team1_avg_strike_rate = []
    team2_avg_strike_rate = []
    team1_total_6s_4s = []
    team2_total_6s_4s = []
    team1_above30_consistancy = []
    team2_above30_consistancy = []
    team1_dots = []
    team2_dots = []
    team1_wicketsbyballs = []
    team2_wicketsbyballs = []
    team1_avg_economy = []
    team2_avg_economy = []
    team1_total_6s_4s_bowl = []
    team1_total_wide_noball = []
    team2_total_6s_4s_bowl = []
    team2_total_wide_noball = []
    team1_total_player_of_match = []
    team2_total_player_of_match = []
    team2_player_wickets = []
    team1_player_wickets = []




    for player in team1_players:

        player_strike_rate = []
        player_runs = []
        player_6s_4s = []
        player_economy = []
        player_wicketsbyball = []
        player_wickets = []
        player_dots = []
        player_6s_4s_bowl = []
        player_wides_noballs = []


        team1_total_player_of_match.append(df_match[df_match['player_of_the_match_id'] == player].shape[0])



        filtered_df = df_batsman[df_batsman['batsman_id'] == player]

        if filtered_df.shape[0] > 0:
 #           filtered_df = filtered_df[filtered_df['match_dt'] < match_date]

 #           if filtered_df.shape[0] > 0:

            for j in range(filtered_df.shape[0]):
                player_6s_4s.append(list(filtered_df["Sixes"])[j])
                player_6s_4s.append(list(filtered_df["Fours"])[j])
                player_strike_rate.append(list(filtered_df["strike_rate"])[j])
                player_runs.append(list(filtered_df["runs"])[j])
                

        filtered_df1 = df_bowler[df_bowler['bowler_id'] == player]

        if filtered_df1.shape[0] > 0:
#            filtered_df1 = filtered_df1[filtered_df1['match_dt'] < match_date]

#            if filtered_df1.shape[0] > 0:

            for k in range(filtered_df1.shape[0]):
                player_economy.append(list(filtered_df1["economy"])[k])
                player_dots.append((list(filtered_df1["dots"])[k]*10)/list(filtered_df1["balls_bowled"])[k])
                player_wicketsbyball.append((list(filtered_df1["wicket_count"])[k]*10)/list(filtered_df1["balls_bowled"])[k])
                player_wickets.append(list(filtered_df1["wicket_count"])[k])
                player_6s_4s_bowl.append(list(filtered_df1["Fours"])[k])
                player_6s_4s_bowl.append(list(filtered_df1["Sixes"])[k])
                player_wides_noballs.append(list(filtered_df1["wides"])[k])
                player_wides_noballs.append(list(filtered_df1["noballs"])[k])

        if len(player_strike_rate) != 0:
            player_avg_strike_rate = np.nanmean(np.array(player_strike_rate))
            team1_avg_strike_rate.append(player_avg_strike_rate)
        
        if len(player_runs) != 0:
            player_avg_runs = np.nanmean(np.array(player_runs))
            team1_avg_runs.append(player_avg_runs)

            new_runs = [30 if x>29 else x for x in player_runs]
            cv1 =[x - 30 for x in new_runs]
            cv1 = [x*x for x in cv1]
            cv1 = np.sqrt(np.nanmean(np.array(cv1)))
            team1_above30_consistancy.append(cv1)


        player_total_6s4s = np.nansum(np.array(player_6s_4s))
        team1_total_6s_4s.append(player_total_6s4s)

        player_total_6s4s_bowl = np.nansum(np.array(player_6s_4s_bowl))
        team1_total_6s_4s_bowl.append(player_total_6s4s_bowl)

        player_total_wide_noball = np.nansum(np.array(player_wides_noballs))
        team1_total_wide_noball.append(player_total_wide_noball)

        player_total_wickets = np.nansum(np.array(player_wickets))
        team1_player_wickets.append(player_total_wickets)

        if len(player_economy) != 0:
            player_avg_economy = np.nanmean(np.array(player_economy))
            team1_avg_economy.append(player_avg_economy)

        if len(player_wicketsbyball) != 0:
            player_avg_wicketsbyball = np.nanmean(np.array(player_wicketsbyball))
            team1_wicketsbyballs.append(player_avg_wicketsbyball)

        if len(player_dots) != 0:
            player_avg_dotsbyball = np.nanmean(np.array(player_dots))
            team1_dots.append(player_avg_dotsbyball)



    for player in team2_players:

        player_strike_rate = []
        player_runs = []
        player_6s_4s = []
        player_economy = []
        player_wicketsbyball = []
        player_wickets = []
        player_dots = []
        player_6s_4s_bowl = []
        player_wides_noballs = []

        team2_total_player_of_match.append(df_match[df_match['player_of_the_match_id'] == player].shape[0])

        filtered_df = df_batsman[df_batsman['batsman_id'] == player]

        if filtered_df.shape[0] > 0:
    #        filtered_df = filtered_df[filtered_df['match_dt'] < match_date]

    #        if filtered_df.shape[0] > 0:

            for j in range(filtered_df.shape[0]):
                player_6s_4s.append(list(filtered_df["Sixes"])[j])
                player_6s_4s.append(list(filtered_df["Fours"])[j])
                player_strike_rate.append(list(filtered_df["strike_rate"])[j])
                player_runs.append(list(filtered_df["runs"])[j])

            

        filtered_df1 = df_bowler[df_bowler['bowler_id'] == player]

        if filtered_df1.shape[0] > 0:
        #    filtered_df1 = filtered_df1[filtered_df1['match_dt'] < match_date]

        #    if filtered_df1.shape[0] > 0:

            for k in range(filtered_df1.shape[0]):
                player_economy.append(list(filtered_df1["economy"])[k])
                player_dots.append((list(filtered_df1["dots"])[k]*10)/list(filtered_df1["balls_bowled"])[k])
                player_wicketsbyball.append((list(filtered_df1["wicket_count"])[k]*10)/list(filtered_df1["balls_bowled"])[k])
                player_wickets.append(list(filtered_df1["wicket_count"])[k])
                player_6s_4s_bowl.append(list(filtered_df1["Fours"])[k])
                player_6s_4s_bowl.append(list(filtered_df1["Sixes"])[k])
                player_wides_noballs.append(list(filtered_df1["wides"])[k])
                player_wides_noballs.append(list(filtered_df1["noballs"])[k])

        if len(player_strike_rate) != 0:
            player_avg_strike_rate = np.nanmean(np.array(player_strike_rate))
            team2_avg_strike_rate.append(player_avg_strike_rate)
        
        if len(player_runs) != 0:
            player_avg_runs = np.nanmean(np.array(player_runs))
            team2_avg_runs.append(player_avg_runs)

            new_runs = [30 if x>29 else x for x in player_runs]
            cv1 =[x - 30 for x in new_runs]
            cv1 = [x*x for x in cv1]
            cv1 = np.sqrt(np.nanmean(np.array(cv1)))
            team2_above30_consistancy.append(cv1)


        player_total_6s4s = np.nansum(np.array(player_6s_4s))
        team2_total_6s_4s.append(player_total_6s4s)

        player_total_6s4s_bowl = np.nansum(np.array(player_6s_4s_bowl))
        team2_total_6s_4s_bowl.append(player_total_6s4s_bowl)

        player_total_wide_noball = np.nansum(np.array(player_wides_noballs))
        team2_total_wide_noball.append(player_total_wide_noball)

        player_total_wickets = np.nansum(np.array(player_wickets))
        team2_player_wickets.append(player_total_wickets)

        if len(player_economy) != 0:
            player_avg_economy = np.nanmean(np.array(player_economy))
            team2_avg_economy.append(player_avg_economy)

        if len(player_wicketsbyball) != 0:
            player_avg_wicketsbyball = np.nanmean(np.array(player_wicketsbyball))
            team2_wicketsbyballs.append(player_avg_wicketsbyball)

        if len(player_dots) != 0:
            player_avg_dotsbyball = np.nanmean(np.array(player_dots))
            team2_dots.append(player_avg_dotsbyball)

    team1_avg_strike_rate = [x-125 if x>125 else 0 for x in team1_avg_strike_rate]
    team1_avg_strike_rate = [x*x for x in team1_avg_strike_rate]

    result["team1_top3_bat_strike_rate"][i] = np.sqrt(np.nanmean(np.array(team1_avg_strike_rate)))
#--------------------------------------------------------------------------------------------
    team2_avg_strike_rate = [x-125 if x>125 else 0 for x in team2_avg_strike_rate]
    team2_avg_strike_rate = [x*x for x in team2_avg_strike_rate]

    result["team2_top3_bat_strike_rate"][i] = np.sqrt(np.nanmean(np.array(team2_avg_strike_rate)))
#------------------------------------------------------------------------------------

    team1_avg_runs = [x-25 if x>25 else 0 for x in team1_avg_runs]
    team1_avg_runs = [x*x for x in team1_avg_runs]

    result["team1_top3_bat_run"][i] = np.sqrt(np.nanmean(np.array(team1_avg_runs)))

#------------------------------------------------------------------------------------------

    team2_avg_runs = [x-25 if x>25 else 0 for x in team2_avg_runs]
    team2_avg_runs = [x*x for x in team2_avg_runs]

    result["team2_top3_bat_run"][i] = np.sqrt(np.nanmean(np.array(team2_avg_runs)))

#-----------------------------------------------------------------------------------------
    result["team1_top3_bat_6s+4s"][i] = np.nanmean(np.array(sorted(team1_total_6s_4s, reverse=True)))
    result["team2_top3_bat_6s+4s"][i] = np.nanmean(np.array(sorted(team2_total_6s_4s, reverse=True)))

    result["team1_top3_bowl_6s+4s"][i] = np.nanmean(np.array(sorted(team1_total_6s_4s_bowl, reverse=False)))
    result["team2_top3_bowl_6s+4s"][i] = np.nanmean(np.array(sorted(team2_total_6s_4s_bowl, reverse=False)))

    result["team1_top3_wide_noball"][i] = np.nanmean(np.array(sorted(team1_total_wide_noball, reverse=False)))
    result["team2_top3_wide_noball"][i] = np.nanmean(np.array(sorted(team2_total_wide_noball, reverse=False)))


#------------------------------------------------------------------------------------
    if len(team1_above30_consistancy) > 2:
        result["team1_above30_consistancy"][i] = np.nanmean(np.array(sorted(team1_above30_consistancy, reverse=False)))
    else:
        result["team1_above30_consistancy"][i] = np.nanmean(np.array(team1_above30_consistancy))
#------------------------------------------------------------------------------------------
    if len(team2_above30_consistancy) > 2:
        result["team2_above30_consistancy"][i] = np.nanmean(np.array(sorted(team2_above30_consistancy, reverse=False)))
    else:
        result["team2_above30_consistancy"][i] = np.nanmean(np.array(team2_above30_consistancy))

#-------------------------------------------------------------------------------------------------------------------------
    team1_avg_economy = [0 if x>7.5 else x for x in team1_avg_economy]
    team1_avg_economy = [x*x for x in team1_avg_economy]

    result["team1_top3_avg_economy"][i] = np.sqrt(np.nanmean(np.array(team1_avg_economy)))

#----------------------------------------------------------------------------------------------------------
    team2_avg_economy = [0 if x>7.5 else x for x in team2_avg_economy]
    team2_avg_economy = [x*x for x in team2_avg_economy]

    result["team2_top3_avg_economy"][i] = np.sqrt(np.nanmean(np.array(team2_avg_economy)))


#-------------------------------------------------------------------------------------------
    if len(team2_wicketsbyballs) > 2:
        result["team2_top3_wickets/ball"][i] = np.nanmean(np.array(sorted(team2_wicketsbyballs, reverse=True)))
    else:
        result["team2_top3_wickets/ball"][i] = np.nanmean(np.array(team2_wicketsbyballs))

#-----------------------------------------------------------------------------------------------------------------
    if len(team1_wicketsbyballs) > 2:
        result["team1_top3_wickets/ball"][i] = np.nanmean(np.array(sorted(team1_wicketsbyballs, reverse=True)))
    else:
        result["team1_top3_wickets/ball"][i] = np.nanmean(np.array(team1_wicketsbyballs))

#---------------------------------------------------------------------------------------------------------
    if len(team1_dots) > 2:
        result["team1_top3_bowl_dots"][i] = np.nanmean(np.array(sorted(team1_dots, reverse=True)))
    else:
        result["team1_top3_bowl_dots"][i] = np.nanmean(np.array(team1_dots))

#-------------------------------------------------------------------------------------------------------
    if len(team2_dots) > 2:
        result["team2_top3_bowl_dots"][i] = np.nanmean(np.array(sorted(team2_dots, reverse=True)))
    else:
        result["team2_top3_bowl_dots"][i] = np.nanmean(np.array(team2_dots))

#--------------------------------------------------------------------------------------------------------

    result["player_of_the_match_team1"][i] = np.nansum(np.array(team1_total_player_of_match))
    result["player_of_the_match_team2"][i] = np.nansum(np.array(team2_total_player_of_match))

    result["team1_player_wickets"][i] = np.nansum(np.array(team1_player_wickets))
    result["team2_player_wickets"][i] = np.nansum(np.array(team2_player_wickets))

In [ ]:
result

In [ ]:
result = result.drop(["team1_roster_ids", "team2_roster_ids"], axis=1)

In [ ]:
result

In [ ]:
match_id = list(result['match id'])
team1_id = list(result['team1_id'])
team2_id = list(result['team2_id'])

venue = list(result['venue'])
city = list(result['city'])
match_date = list(result['match_dt'])


features = result.drop(['match id', 'team1_id', 'team2_id', 'match_dt', 'city', 'venue'], axis= 1)


#features = features.drop(columns='match id')

# Initialize the IterativeImputer with RandomForestRegressor
#imputer = IterativeImputer(estimator=RandomForestRegressor(), random_state=0)

# Fit the imputer and transform the features
imputed_features = imputer.transform(features)

# Convert the result back to a DataFrame
features_imputed = pd.DataFrame(imputed_features, columns=features.columns)


In [ ]:
features_imputed['match id'] = match_id
features_imputed['team1_id'] = team1_id
features_imputed['team2_id'] = team2_id

features_imputed["venue"] = venue
features_imputed["city"] = city
features_imputed["match_dt"] = match_date

In [ ]:
features_imputed

In [ ]:
for i in range(features_imputed.shape[0]):

    features_imputed["match_dt"][i] = features_imputed["match_dt"][i].month

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder


label_encoder = LabelEncoder()
features_imputed["venue"] = label_encoder.fit_transform(features_imputed["venue"])
features_imputed["city"] = label_encoder.fit_transform(features_imputed["city"])

In [ ]:
features_imputed

In [ ]:
features_imputed.to_csv("sub_sample_all_feat222.csv", index=False)

In [ ]:
features_imputed